<a href="https://colab.research.google.com/github/Deji01/LLMs/blob/main/Newspaper_Articles_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain openai tiktoken newspaper3k deeplake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 kB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.

In [2]:
from getpass import getpass
import openai
import os

os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY")
openai.api_key  =  os.environ["OPENAI_API_KEY"]

OPENAI_API_KEY··········


In [ ]:
import requests
from newspaper import Article

In [42]:
headers =  {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

In [43]:
article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"


In [44]:
session = requests.Session()

In [45]:
try:
  response = session.get(article_url, headers=headers, timeout=10)

  if response.status_code == 200:
    article = Article(article_url)
    article.download()
    article.parse()

    print(f"Title: {article.title}")
    print(f"Text: {article.text}")

  else:
    print(f"Failed to fetch article at {article_url}")
except Exeption as e:
   print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entire

In [46]:
from langchain.schema import HumanMessage

In [47]:
article_title = article.title
article_text = article.text

In [49]:
article.text

"Ryan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)\n\nMeta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.\n\nThe supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.\n\nRSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.\n\n“We hope RSC will help us build entirely new AI systems that can, for example, power real-time v

In [83]:
# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into an organized data structure. Here are a few examples of how you've done this in the past:

Example 1:

{{
  "title" : "The Effects of Climate Change",
  "summary" : [ "Climate change is causing a rise in global temperatures.", "This leads to melting ice caps and rising sea levels.", " Resulting in more frequent and severe weather conditions." ]
}}

Example 2:

{{
  "title" : "The Evolution of Artificial Intelligence",
  "summary" :[ "Artificial Intelligence (AI) has developed significantly over the past decade.", "AI is now used in multiple fields such as healthcare, finance, and transportation.", "The future of AI is promising but requires careful regulation." ]
}}

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in dictionary containing the title and list of summary.
"""

In [84]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List

# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [98]:
from langchain import PromptTemplate

In [85]:
prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [86]:
# prompt = template.format(article_title=article.title, article_text=article.text)
# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format_prompt(article_title=article_title, article_text=article_text)

In [87]:
formatted_prompt

StringPromptValue(text='\nAs an advanced AI, you\'ve been tasked to summarize online articles into an organized data structure. Here are a few examples of how you\'ve done this in the past:\n\nExample 1:\n\n{\n  "title" : "The Effects of Climate Change", \n  "summary" : [ "Climate change is causing a rise in global temperatures.", "This leads to melting ice caps and rising sea levels.", " Resulting in more frequent and severe weather conditions." ]\n}\n\nExample 2:\n\n{ \n  "title" : "The Evolution of Artificial Intelligence", \n  "summary" :[ "Artificial Intelligence (AI) has developed significantly over the past decade.", "AI is now used in multiple fields such as healthcare, finance, and transportation.", "The future of AI is promising but requires careful regulation." ]\n}\n\nNow, here\'s the article you need to summarize:\n\n==================\nTitle: Meta claims its new AI supercomputer will set records\n\nRyan is a senior editor at TechForge Media with over a decade of experienc

In [89]:
messages = [HumanMessage(content=formatted_prompt.to_string())]

In [90]:
from langchain.chat_models import ChatOpenAI

In [91]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [92]:
# generate summary
summary = chat(messages)
print(summary.content)

{
  "title" : "Meta unveils AI supercomputer for training large models",
  "summary" : [ "Meta (formerly Facebook) has introduced the AI Research SuperCluster (RSC), an AI supercomputer that aims to be the fastest in the world.", "The RSC is designed to train large natural language processing (NLP) and computer vision models.", "Once complete, it is expected to be capable of training models with trillions of parameters.", "Meta plans to use the RSC to develop AI systems for real-time voice translations and applications in the metaverse.", "The RSC is estimated to be 20x faster than Meta's current clusters and will improve training times for large-scale NLP workflows." ]
}


In [93]:
parsed_output = parser.parse(summary.content)
print(parsed_output)

title='Meta unveils AI supercomputer for training large models' summary=['Meta (formerly Facebook) has introduced the AI Research SuperCluster (RSC), an AI supercomputer that aims to be the fastest in the world.', 'The RSC is designed to train large natural language processing (NLP) and computer vision models.', 'Once complete, it is expected to be capable of training models with trillions of parameters.', 'Meta plans to use the RSC to develop AI systems for real-time voice translations and applications in the metaverse.', "The RSC is estimated to be 20x faster than Meta's current clusters and will improve training times for large-scale NLP workflows."]


In [94]:
parsed_output.dict()

{'title': 'Meta unveils AI supercomputer for training large models',
 'summary': ['Meta (formerly Facebook) has introduced the AI Research SuperCluster (RSC), an AI supercomputer that aims to be the fastest in the world.',
  'The RSC is designed to train large natural language processing (NLP) and computer vision models.',
  'Once complete, it is expected to be capable of training models with trillions of parameters.',
  'Meta plans to use the RSC to develop AI systems for real-time voice translations and applications in the metaverse.',
  "The RSC is estimated to be 20x faster than Meta's current clusters and will improve training times for large-scale NLP workflows."]}

In [95]:
parsed_output.json()

'{"title": "Meta unveils AI supercomputer for training large models", "summary": ["Meta (formerly Facebook) has introduced the AI Research SuperCluster (RSC), an AI supercomputer that aims to be the fastest in the world.", "The RSC is designed to train large natural language processing (NLP) and computer vision models.", "Once complete, it is expected to be capable of training models with trillions of parameters.", "Meta plans to use the RSC to develop AI systems for real-time voice translations and applications in the metaverse.", "The RSC is estimated to be 20x faster than Meta\'s current clusters and will improve training times for large-scale NLP workflows."]}'

In [96]:
parsed_output.title

'Meta unveils AI supercomputer for training large models'

In [97]:
parsed_output.summary

['Meta (formerly Facebook) has introduced the AI Research SuperCluster (RSC), an AI supercomputer that aims to be the fastest in the world.',
 'The RSC is designed to train large natural language processing (NLP) and computer vision models.',
 'Once complete, it is expected to be capable of training models with trillions of parameters.',
 'Meta plans to use the RSC to develop AI systems for real-time voice translations and applications in the metaverse.',
 "The RSC is estimated to be 20x faster than Meta's current clusters and will improve training times for large-scale NLP workflows."]